In [1]:
import numpy as np
import pandas as pd
import os
import scipy.io
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import GridSearchCV
import scipy.io
import time
import pyreadr

In [2]:
# from google.colab import drive
# drive.mount('/gdrive')

Mounted at /gdrive


In [6]:
train_dat = pyreadr.read_r('../output/feature_train.RData')
test_dat = pyreadr.read_r('../output/feature_test.RData')

train_dat = list(train_dat.items())[0][1]
test_dat = list(test_dat.items())[0][1]

In [7]:
x_train = train_dat.iloc[:,:-1]
y_train = train_dat.iloc[:,-1]
x_test = test_dat.iloc[:,:-1]
y_test = test_dat.iloc[:,-1]

In [8]:
best_model = Sequential()
best_model.add(BatchNormalization(input_shape=[6006], momentum=0.80))
best_model.add(Dense(1024, activation='relu'))
best_model.add(Dropout(0.5))
best_model.add(Dense(512, activation='relu'))
best_model.add(Dropout(0.5))
best_model.add(Dense(256, activation='relu'))
best_model.add(Dropout(0.3))
best_model.add(Dense(128, activation='relu'))
best_model.add(Dropout(0.3))
best_model.add(Dense(64, activation='relu'))
best_model.add(Dropout(0.2))
best_model.add(Dense(32, activation='relu'))
best_model.add(Dropout(0.2))
best_model.add(Dense(16, activation='relu'))
best_model.add(Dropout(0.1))
best_model.add(Dense(8, activation='relu'))
best_model.add(Dropout(0.1))
best_model.add(Dense(1, activation='sigmoid'))

In [15]:
#compile model
best_model.compile(loss='binary_crossentropy', 
                   optimizer='adam', 
                   metrics=['accuracy', 'AUC'])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=20,
    mode='max',
    restore_best_weights=True)

In [16]:
y_train = y_train.astype('int')

In [17]:
start_time = time.time()
best_model.fit(x_train, 
               y_train, 
               batch_size=32, 
               epochs=100, 
               validation_split=0.15,
               callbacks = [early_stopping])
print('Training on 2400 images with train-val split of 85-15 takes %s seconds.'%round((time.time()-start_time),3))

Epoch 1/100
64/64 [==============================] - 7s 92ms/step - loss: 0.3380 - accuracy: 0.8077 - auc: 0.8903 - val_loss: 0.4885 - val_accuracy: 0.7944 - val_auc: 0.8038
Epoch 2/100
64/64 [==============================] - 6s 88ms/step - loss: 0.3240 - accuracy: 0.8073 - auc: 0.8992 - val_loss: 0.4833 - val_accuracy: 0.7944 - val_auc: 0.7949
Epoch 3/100
64/64 [==============================] - 6s 86ms/step - loss: 0.3339 - accuracy: 0.7944 - auc: 0.8936 - val_loss: 0.4328 - val_accuracy: 0.7944 - val_auc: 0.8089
Epoch 4/100
64/64 [==============================] - 5s 85ms/step - loss: 0.3494 - accuracy: 0.7951 - auc: 0.8785 - val_loss: 0.4869 - val_accuracy: 0.7944 - val_auc: 0.7917
Epoch 5/100
64/64 [==============================] - 6s 86ms/step - loss: 0.3670 - accuracy: 0.7991 - auc: 0.8680 - val_loss: 0.4992 - val_accuracy: 0.7944 - val_auc: 0.7797
Epoch 6/100
64/64 [==============================] - 6s 86ms/step - loss: 0.3278 - accuracy: 0.8136 - auc: 0.8849 - val_loss: 0.48

In [20]:
y_test = y_test.astype('int')

In [21]:
start_time = time.time()
best_model.evaluate(x_test, y_test, batch_size=32, verbose=1)
print('Prediction on 600 images takes %s seconds.'%round((time.time()-start_time),3))

19/19 [==============================] - 0s 10ms/step - loss: 0.8101 - accuracy: 0.8033 - auc: 0.8060
Prediction on 600 images takes 0.352 seconds.


In [22]:
tf.keras.models.save_model(best_model, '../output/nn_model', save_format='h5')
best_model = tf.keras.models.load_model('../output/nn_model')

In [24]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 6006)              24024     
_________________________________________________________________
dense (Dense)                (None, 1024)              6151168   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0